In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103

## 1. Análisis de Procedimientos a Nivel País

En este análisis se quieren responder las siguiente preguntas:

1. ¿Cómo es el ranking de los hospitales para cada uno de los procedimientos realizads?
2. ¿Quién es el Hospital que realiza más de X procedimientos?
3. ¿El procedimiento X es realizado en todo el país? ¿En qué partes en específico se realiza?

Para responder esta pregunta se utilizará la base de datos de Egresos de Hospitales adscritos
a GRD entre 2019 y 2021.


In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

In [5]:
display(df_procesada.head(10))

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA
0,118100,1314867.0,MUJER,1988-06-19,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,1.0,146102.0,0.5744,2.0,1.0,NaN,5.0,2019.0,3.0,2019-3
1,118100,1354418.0,MUJER,1988-03-27,OTRO,CONCEPCION,SAN PEDRO DE LA PAZ,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,NaN,134161.0,0.2951,1.0,1.0,NaN,4.0,2019.0,3.0,2019-3
2,118100,239861.0,MUJER,2017-06-17,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,64182.0,0.6736,2.0,2.0,NaN,6.0,2019.0,3.0,2019-3
3,118100,330326.0,HOMBRE,2017-12-04,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) A,CONCEPCIÓN,...,NaN,64172.0,0.5475,2.0,2.0,NaN,2.0,2019.0,3.0,2019-3
4,118100,1369293.0,MUJER,1995-12-06,OTRO,CONCEPCION,FLORIDA,CHILE,FONASA INSTITUCIONAL - (MAI) C,CONCEPCIÓN,...,NaN,134141.0,0.3107,1.0,1.0,NaN,6.0,2019.0,3.0,2019-3
5,118100,469385.0,MUJER,1942-06-29,OTRO,CONCEPCION,CHIGUAYANTE,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,114103.0,1.1799,3.0,3.0,NaN,47.0,2019.0,3.0,2019-3
6,118100,1146635.0,MUJER,1951-07-14,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,44163.0,1.0328,3.0,3.0,NaN,43.0,2019.0,3.0,2019-3
7,118100,93557.0,HOMBRE,1938-03-04,OTRO,CONCEPCION,LOTA,CHILE,FONASA INSTITUCIONAL - (MAI) D,CONCEPCIÓN,...,1.0,11101.0,1.8731,1.0,1.0,NaN,41.0,2019.0,3.0,2019-3
8,118100,674943.0,MUJER,1962-08-21,OTRO,CONCEPCION,SANTA JUANA,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,1.0,61402.0,1.3078,2.0,2.0,HOSPITAL CLORINDA AVELLO (SANTA JUANA),33.0,2019.0,3.0,2019-3
9,118100,1079955.0,HOMBRE,1944-12-07,OTRO,CONCEPCION,CONCEPCIÓN,CHILE,FONASA INSTITUCIONAL - (MAI) B,CONCEPCIÓN,...,NaN,14133.0,1.6687,3.0,3.0,NaN,31.0,2019.0,3.0,2019-3


Los procedimientos en esta base de datos se encuentran en las columnas "PROCEDIMIENTOSX", donde X
va desde 1 hasta 30. Para analizar esta base de forma más expedita, se transformará de formato
ancho a formato largo, utilizando como identificador a las columnas ["COD_HOSPITAL", "ANIO_EGRESO",
"CIP_ENCRIPTADO", "DIAGNOSTICO1"]. Cabe destacar que la columna "CIP_ENCRIPTADO" hace referencia al
RUT del paciente.


In [6]:
procedimientos_formato_long = pd.melt(
    df_procesada,
    id_vars=[
        "COD_HOSPITAL",
        "ANIO_EGRESO",
        "TIPO_ACTIVIDAD",
        "CIP_ENCRIPTADO",
        "DIAGNOSTICO1",
    ],
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [7]:
var_categorica_a_contar = "procedimiento"
desglose_a_obtener = ["ANIO_EGRESO", "COD_HOSPITAL"]
grupo_ranking = ["ANIO_EGRESO", "procedimiento"]

conteo_y_ranking_procedimientos = (
    build_features.calcular_conteo_y_ranking_sobre_variable_categorica(
        procedimientos_formato_long,
        var_categorica_a_contar,
        desglose_a_obtener,
        grupo_ranking,
    )
)

In [8]:
display(conteo_y_ranking_procedimientos.head(20))

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento
180193,2021.0,113180,99.99,4,1,2,18,0.22
160738,2021.0,108100,99.99,2,2,1,18,0.11
148451,2021.0,103100,99.99,1,3,0,18,0.06
152391,2021.0,105100,99.99,1,4,0,18,0.06
166209,2021.0,110100,99.99,1,5,0,18,0.06
167455,2021.0,110120,99.99,1,6,0,18,0.06
179376,2021.0,113130,99.99,1,7,0,18,0.06
182902,2021.0,114101,99.99,1,8,0,18,0.06
187970,2021.0,115107,99.99,1,9,0,18,0.06
188758,2021.0,115110,99.99,1,10,0,18,0.06


El resultado muestra el ranking para todos los procedimientos asignados en el país en cada uno de
los años analizados (2019 a 2021).

A modo de ejemplo, se ilustra el ranking en el año 2021 para el procedimiento 99.99. En este caso,
se observa que los hospitales con código 108100 y 113180 están en los primeros lugares, realizando
2 procedimientos cada uno.

Ahora, si se filtra por el hospital del tórax, y sólo los procedimientos donde el hospital sea
ranking 1, entonces se obtendrán procedimientos altamente relevantes para el país.


In [9]:
cie_9 = pd.read_excel("../data/external/CIE-9.xlsx", dtype={"Código": str})
esquema_hospitales = pd.read_excel(
    "../data/external/Esquema_Registro-2023.xls", sheet_name=4, header=7
).dropna(how="all", axis=1)

In [10]:
ranking_con_nombre_procedimientos = pd.merge(
    conteo_y_ranking_procedimientos, cie_9, how="left", left_on="procedimiento", right_on="Código"
)
ranking_con_nombre_procedimientos_y_establecimientos = ranking_con_nombre_procedimientos.merge(
    esquema_hospitales, how="left", left_on="COD_HOSPITAL", right_on="Código nuevo Establecimiento"
)

In [11]:
display(ranking_con_nombre_procedimientos_y_establecimientos)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento,Código,Descripción,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
0,2021.0,113180,99.99,4,1,2,18,0.22,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,113180,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital El Pino (Santiago, San Bernardo)",13401,13
1,2021.0,108100,99.99,2,2,1,18,0.11,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,108100,Servicio de Salud Aconcagua,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Camilo de San Felipe,5701,5
2,2021.0,103100,99.99,1,3,0,18,0.06,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,103100,Servicio de Salud Antofagasta,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Dr. Leonardo Guzmán (Antofagasta),2101,2
3,2021.0,105100,99.99,1,4,0,18,0.06,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,105100,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Juan de Dios (La Serena),4101,4
4,2021.0,110100,99.99,1,5,0,18,0.06,99.99,PROCEDIMIENTO MISCELANEO.OTRO,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,110100,Servicio de Salud Metropolitano Occidente,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital San Juan de Dios (Santiago, Santiago)",13101,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214751,2019.0,105101,00.01,65,1,52,80,0.81,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,105101,Servicio de Salud Coquimbo,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital San Pablo (Coquimbo),4102,4
214752,2019.0,106100,00.01,13,2,12,80,0.16,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,106100,Servicio de Salud Valparaíso San Antonio,Perteneciente,1,Hospital (Alta Complejidad),4.0,Hospital Carlos Van Buren (Valparaíso),5101,5
214753,2019.0,113100,00.01,1,3,0,80,0.01,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,113100,Servicio de Salud Metropolitano Sur,Perteneciente,1,Hospital (Alta Complejidad),4.0,"Hospital Barros Luco Trudeau (Santiago, San Mi...",13130,13
214754,2019.0,120101,00.01,1,4,-1,80,0.01,00.01,ULTRASONIDO TERAPEUTICO.VASO CABEZA Y CUELLO,...,00. (00 00) PROCEDIMIENTOS E INTERVENCIONES NO...,120101,Servicio de Salud Biobío,Perteneciente,1,Hospital (Alta Complejidad),4.0,Complejo Asistencial Dr. Víctor Ríos Ruiz (Los...,8301,8


Ahora se puede observar el nombre del procedimiento analizado y el nombre del hospital visto.


In [12]:
conteo_procedimientos_torax = ranking_con_nombre_procedimientos_y_establecimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX"
).sort_values(["ANIO_EGRESO", "conteo_procedimiento"], ascending=False)

mejores_procedimientos_torax = conteo_procedimientos_torax.query(
    "ranking_conteo_procedimiento == 1"
)

display(mejores_procedimientos_torax)

,ANIO_EGRESO,COD_HOSPITAL,procedimiento,conteo_procedimiento,ranking_conteo_procedimiento,diferencias_procedimiento,totales_procedimiento,proporcion_procedimiento,Código,Descripción,...,Capítulo,Código nuevo Establecimiento,SEREMI / Servicio de Salud,Pertenencia al SNSS,Código tipo Pertenencia,TipoEstablecimiento,Código tipo establecimiento,Nombre Oficial,Código Comuna,Código Región
54295,2021.0,112103,39.61,626,1,68,3184,0.20,39.61,CIRCULACION EXTRACORPOREA PARA CIRUGIA CORAZON...,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
54251,2021.0,112103,39.64,584,1,246,2020,0.29,39.64,MARCAPASOS CORAZON INTRAOPERATORIO,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
52933,2021.0,112103,42.23,391,1,342,607,0.64,42.23,ESOFAGOSCOPIA OTRA,...,09. (42 54) OPERACIONES SOBRE EL APARATO DIGES...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
57293,2021.0,112103,37.22,280,1,22,801,0.35,37.22,CATETERISMO CORAZON LADO IZQUIERDO,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
13242,2021.0,112103,89.46,278,1,82,1355,0.21,89.46,COMPROBACION FORMA ONDA ARTEFACTO MARCAPASOS C...,...,16. (87 99) PROCEDIMIENTOS DIAGNOSTICOS Y TERA...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200802,2019.0,112103,34.89,2,1,1,6,0.33,34.89,OPERACION DIAFRAGMA.OTRA,...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
201716,2019.0,112103,33.49,2,1,1,10,0.20,33.49,OPERACION PULMON REPARADORA Y PLASTICA OTRA,...,06. (30 34) OPERACIONES SOBRE EL APARATO RESPI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
211026,2019.0,112103,07.81,2,1,1,10,0.20,07.81,ESCISION TIMO PARCIAL OTRA,...,02. (06 07) OPERACIONES SOBRE EL SISTEMA ENDOC...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13
198540,2019.0,112103,38.46,1,1,0,3,0.33,38.46,RESECCION ARTERIA ABDOMEN CON SUSTITUCION,...,07. (35 39) OPERACIONES SOBRE EL APARATO CARDI...,112103,Servicio de Salud Metropolitano Oriente,Perteneciente,1,Hospital (Alta Complejidad),4.0,Instituto Nacional de Enfermedades Respiratori...,13123,13


Los resultados muestran que el tórax en el año 2021 fue altamente relevante en los procedimientos
39.61, 39.64, 42.23, entre otros. Además, se puede observar la diferencia que tiene el Tórax
con los siguientes establecimientos de salud y el total de procedimientos realizados en el país.


Ahora, se quiere saber la cantidad de procedimientos, pero sabiendo si fue realizado de forma
ambulatoria, hospitalizada, cirugia mayor ambulatoria, etc...


In [14]:
var_categorica_a_contar = "procedimiento"
desglose_a_obtener = ["ANIO_EGRESO", "COD_HOSPITAL", "TIPO_ACTIVIDAD"]
grupo_ranking = ["ANIO_EGRESO", "TIPO_ACTIVIDAD", "procedimiento"]

conteo_y_ranking_procedimientos_por_tipoalta = (
    build_features.calcular_conteo_y_ranking_sobre_variable_categorica(
        procedimientos_formato_long,
        var_categorica_a_contar,
        desglose_a_obtener,
        grupo_ranking,
    )
)

In [28]:
with pd.ExcelWriter("../data/interim/procedimientos_grd.xlsx") as file:
    conteo_procedimientos_torax.to_excel(file, sheet_name="procedimientos_torax", index=False)
    conteo_y_ranking_procedimientos.to_excel(file, sheet_name="resumen_nacional", index=False)
    conteo_y_ranking_procedimientos_por_tipoalta.to_excel(
        file, sheet_name="resumen_nacional_tipoalta", index=False
    )

## 2. Obtención de perfiles de pacientes por procedimientos

En este apartado se quieren responder las siguientes preguntas:

1. ¿Cuál es la **mortalidad** de cada uno de los procedimientos?
2. ¿Cuál es el **peso GRD** promedio de cada uno de los procedimientos?
3. ¿Cuál es la **severidad** de cada uno de los procedimientos?
4. Dentro de los pacientes muertos, **¿Cuál es la severidad que tienen?**

Ahora, la base de GRD tiene hasta 30 procedimientos, por lo que en un mismo egreso se pueden
repetir procedimientos. Para reducir la complejidad del análisis, se utilizará solamente el primer
procedimiento para agrupar por severidad y mortalidad. Esto, ya que 1 procedimiento corresponderá
a 1 egreso.

Por lo tanto, la base de datos se debe mantener como está, pero solamente seleccionando las variables
recién mencionadas


In [16]:
vars_de_interes = [
    "COD_HOSPITAL",
    "ANIO_EGRESO",
    "TIPO_ACTIVIDAD",
    "CIP_ENCRIPTADO",
    "TIPOALTA",
    "DIAGNOSTICO1",
    "PROCEDIMIENTO1",
    "IR_29301_PESO",
    "IR_29301_SEVERIDAD",
]

In [17]:
df_para_mortalidad_y_severidad = df_procesada[vars_de_interes]

In [18]:
display(df_para_mortalidad_y_severidad)

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,CIP_ENCRIPTADO,TIPOALTA,DIAGNOSTICO1,PROCEDIMIENTO1,IR_29301_PESO,IR_29301_SEVERIDAD
0,118100,2019.0,HOSPITALIZACIÓN,1314867.0,DOMICILIO,O82.0,74.1,0.5744,2.0
1,118100,2019.0,HOSPITALIZACIÓN,1354418.0,DOMICILIO,O26.9,88.78,0.2951,1.0
2,118100,2019.0,HOSPITALIZACIÓN,239861.0,DOMICILIO,K92.0,87.03,0.6736,2.0
3,118100,2019.0,HOSPITALIZACIÓN,330326.0,DOMICILIO,A08.3,90.92,0.5475,2.0
4,118100,2019.0,HOSPITALIZACIÓN,1369293.0,DOMICILIO,O60.0,75.34,0.3107,1.0
...,...,...,...,...,...,...,...,...,...
2750291,105100,2021.0,HOSPITALIZACIÓN,68882797.0,DOMICILIO,N39.0,57.17,1.6547,3.0
2750292,120101,2021.0,HOSPITALIZACIÓN,97789022.0,DOMICILIO,K40.9,53.00,0.8122,1.0
2750293,114101,2021.0,HOSPITALIZACIÓN,71164179.0,DOMICILIO,Z51.8,00.18,1.0867,2.0
2750294,107100,2021.0,HOSPITALIZACIÓN,78884679.0,DOMICILIO,K80.0,51.23,1.1472,2.0


Ahora que se tiene la base de datos con menos variables, se quiere responder a la pregunta:

¿Cuál es la mortalidad de cada uno de los procedimientos?


In [19]:
desglose_procedimientos = df_para_mortalidad_y_severidad.groupby(
    [
        "COD_HOSPITAL",
        "ANIO_EGRESO",
        "TIPO_ACTIVIDAD",
        "TIPOALTA",
        "PROCEDIMIENTO1",
        "IR_29301_SEVERIDAD",
    ]
).agg(cantidad_procedimientos=("PROCEDIMIENTO1", "count"), peso_grd_medio=("IR_29301_PESO", "mean"))

display(desglose_procedimientos.head(10))

desglose_procedimientos = desglose_procedimientos.reset_index().sort_values(
    ["COD_HOSPITAL", "ANIO_EGRESO", "PROCEDIMIENTO1"]
)

cantidad_procedimientos  \
COD_HOSPITAL ANIO_EGRESO TIPO_ACTIVIDAD                  TIPOALTA  PROCEDIMIENTO1 IR_29301_SEVERIDAD                            
101100       2019.0      CIRUGÍA MAYOR AMBULATORIA (CMA) DOMICILIO 03.92          0.0                                      36   
                                                                   04.11          0.0                                       1   
                                                                   04.43          0.0                                      45   
                                                                   04.81          0.0                                       9   
                                                                   06.31          0.0                                       1   
                                                                   06.7           0.0                                       1   
                                                                   08.01          0.0                                       1   
                                                                   08.20          0.0                                       5   
                                                                   08.22          0.0                                       1   
                                                                   08.33          0.0                                       1   

                                                                                                      peso_grd_medio  
COD_HOSPITAL ANIO_EGRESO TIPO_ACTIVIDAD                  TIPOALTA  PROCEDIMIENTO1 IR_29301_SEVERIDAD                  
101100       2019.0      CIRUGÍA MAYOR AMBULATORIA (CMA) DOMICILIO 03.92          0.0                       0.148200  
                                                                   04.11          0.0                       0.297300  
                                                                   04.43          0.0                       0.304500  
                                                                   04.81          0.0                       0.147722  
                                                                   06.31          0.0                       0.987000  
                                                                   06.7           0.0                       0.552400  
                                                                   08.01          0.0                       0.188500  
                                                                   08.20          0.0                       0.297580  
                                                                   08.22          0.0                       0.273700  
                                                                   08.33          0.0                       0.491900

Los resultados anteriores indican lo siguiente:

El hospital 101100 en el año 2019 tuvo 36 procedimientos 1 03.92 que tuvieron una severidad de 0.
Estos procedimientos fueron realizados en una cirugía mayor ambulatorio (CMA), y la alta de
sus pacientes fueron en DOMICILIO. Dentro de los 36 egresos, estos tuvieron una complejidad
promedio de 0.148200.

Ahora, para obtener la mortalidad de cada procedimiento es necesario consultar los pacientes
que tienen un TIPOALTA igual a FALLECIDO.


In [20]:
display(desglose_procedimientos.query("TIPOALTA == 'FALLECIDO'"))

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,TIPOALTA,PROCEDIMIENTO1,IR_29301_SEVERIDAD,cantidad_procedimientos,peso_grd_medio
1748,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,2.0,2,1.818300
1749,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,3,2.484000
1750,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,01.24,3.0,1,11.704500
1751,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,01.27,2.0,1,1.361400
1752,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,01.31,3.0,1,4.345400
...,...,...,...,...,...,...,...,...
343286,133150,2021.0,HOSPITALIZACIÓN,FALLECIDO,99.26,2.0,1,0.700000
343287,133150,2021.0,HOSPITALIZACIÓN,FALLECIDO,99.26,3.0,2,1.622900
343288,133150,2021.0,HOSPITALIZACIÓN,FALLECIDO,99.29,1.0,1,0.534600
343289,133150,2021.0,HOSPITALIZACIÓN,FALLECIDO,99.29,2.0,8,1.203313


Los resultados anteriores indican que en el Hospital 101100, en el año 2019 fallecieron 2
egresos que se realizaron 00.66 en una hospitalización. Estos egresos tenían una severidad de 2,
con un peso GRD promedio de 1.8183.

A modo de ejemplo, se realizará la consulta en la base de datos y se mostrarán los pacientes
que cumplen los criterios.


In [21]:
df_procesada.query(
    "ANIO_EGRESO == 2019.0 and COD_HOSPITAL == 101100 and "
    "PROCEDIMIENTO1 == '00.66' and IR_29301_SEVERIDAD == 2.0 and TIPOALTA == 'FALLECIDO'"
)

,COD_HOSPITAL,CIP_ENCRIPTADO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,...,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA,ESTANCIA,ANIO_EGRESO,MES_EGRESO,FECHA
182374,101100,1370515.0,HOMBRE,1949-06-06,OTRO,ARICA,ARICA,CHILE,FONASA INSTITUCIONAL - (MAI) D,ARICA,...,4.0,51402.0,1.8183,2.0,2.0,NaN,11.0,2019.0,11.0,2019-11
631299,101100,569058.0,HOMBRE,1937-12-16,OTRO,ARICA,ARICA,CHILE,FONASA INSTITUCIONAL - (MAI) B,ARICA,...,4.0,51402.0,1.8183,2.0,2.0,NaN,3.0,2019.0,11.0,2019-11


Sin embargo, es necesario saber cuánto es el porcentaje de muertos del total.


In [22]:
total_proceds = desglose_procedimientos.groupby(["COD_HOSPITAL", "ANIO_EGRESO", "PROCEDIMIENTO1"])[
    "cantidad_procedimientos"
].transform("sum")

desglose_procedimientos["total_de_procedimientos"] = total_proceds

desglose_procedimientos["ratio_de_procedimientos"] = round(
    desglose_procedimientos["cantidad_procedimientos"]
    / desglose_procedimientos["total_de_procedimientos"],
    2,
)

In [23]:
display(desglose_procedimientos.head(10))

,COD_HOSPITAL,ANIO_EGRESO,TIPO_ACTIVIDAD,TIPOALTA,PROCEDIMIENTO1,IR_29301_SEVERIDAD,cantidad_procedimientos,peso_grd_medio,total_de_procedimientos,ratio_de_procedimientos
414,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.17,1.0,2,0.4149,3,0.67
415,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.17,2.0,1,0.5390,3,0.33
416,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,1.0,15,1.7184,34,0.44
417,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,2.0,12,1.8183,34,0.35
418,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.66,3.0,2,2.4840,34,0.06
1748,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,2.0,2,1.8183,34,0.06
1749,101100,2019.0,HOSPITALIZACIÓN,FALLECIDO,00.66,3.0,3,2.4840,34,0.09
419,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.70,2.0,1,2.2875,1,1.00
420,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.73,1.0,3,1.4477,4,0.75
421,101100,2019.0,HOSPITALIZACIÓN,DOMICILIO,00.73,2.0,1,1.5876,4,0.25


Los resultados anteriores se pueden interpretar de la siguiente manera:

- El hospital 101100, en el año 2019 tuvo 5 procedimientos (y por ende 5 egresos) en donde
  la persona falleció realizandose un procedimiento 00.66. Estos egresos tenían una severidad 2
  y 3, y corresponden al 15% de todos los procedimientos realizados.


In [24]:
desglose_procedimientos = desglose_procedimientos.merge(
    cie_9, how="left", left_on="PROCEDIMIENTO1", right_on="Código"
).merge(
    esquema_hospitales,
    how="left",
    left_on="COD_HOSPITAL",
    right_on="Código nuevo Establecimiento",
)

In [25]:
desglose_procedimientos.to_csv(
    "../data/interim/mortalidad_y_severidad_procedimientos.csv",
    index=False,
    encoding="latin-1",
    sep=";",
    decimal=","
)